In [1]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)
# Use pd.NA for missing values
pd.options.mode.use_inf_as_na = True


In [13]:
# Import and clean data from data-parsed folder
SRC_DIR = './data-parsed/teams/ATL/2022'
TGL_TYPE_CSV = 'tgl_advanced.csv'
TGL_DF = pd.read_csv(f'{SRC_DIR}/{TGL_TYPE_CSV}',header=[0,1]).drop(columns=[('Unnamed: 0_level_0','Unnamed: 0_level_1')])
TGL_DF

Match                                                                        
       G        Date             Boxscores_html_id   Tm            Tm_html_id   
0      1  2021-10-21  /boxscores/202110210ATL.html  ATL  /teams/ATL/2022.html  \
1      2  2021-10-23  /boxscores/202110230CLE.html  ATL  /teams/ATL/2022.html   
2      3  2021-10-25  /boxscores/202110250ATL.html  ATL  /teams/ATL/2022.html   
3      4  2021-10-27  /boxscores/202110270NOP.html  ATL  /teams/ATL/2022.html   
4      5  2021-10-28  /boxscores/202110280WAS.html  ATL  /teams/ATL/2022.html   
5      6  2021-10-30  /boxscores/202110300PHI.html  ATL  /teams/ATL/2022.html   
6      7  2021-11-01  /boxscores/202111010ATL.html  ATL  /teams/ATL/2022.html   
7      8  2021-11-03  /boxscores/202111030BRK.html  ATL  /teams/ATL/2022.html   
8      9  2021-11-04  /boxscores/202111040ATL.html  ATL  /teams/ATL/2022.html   
9     10  2021-11-06  /boxscores/202111060PHO.html  ATL  /teams/ATL/2022.html   
10    11  2021-11-08  /boxscores/202111080GSW.html  ATL  /teams/ATL/2022.html   
11    12  2021-11-09  /boxscores/202111090UTA.html  ATL  /teams/ATL/2022.html   
12    13  2021-11-12  /boxscores/202111120DEN.html  ATL  /teams/ATL/2022.html   
13    14  2021-11-14  /boxscores/202111140ATL.html  ATL  /teams/ATL/2022.html   
14    15  2021-11-15  /boxscores/202111150ATL.html  ATL  /teams/ATL/2022.html   
..   ...         ...                           ...  ...                   ...   
67    68  2022-03-14  /boxscores/202203140ATL.html  ATL  /teams/ATL/2022.html   
68    69  2022-03-16  /boxscores/202203160CHO.html  ATL  /teams/ATL/2022.html   
69    70  2022-03-18  /boxscores/202203180ATL.html  ATL  /teams/ATL/2022.html   
70    71  2022-03-20  /boxscores/202203200ATL.html  ATL  /teams/ATL/2022.html   
71    72  2022-03-22  /boxscores/202203220NYK.html  ATL  /teams/ATL/2022.html   
72    73  2022-03-23  /boxscores/202203230DET.html  ATL  /teams/ATL/2022.html   
73    74  2022-03-25  /boxscores/202203250ATL.html  ATL  /teams/ATL/2022.html   
74    75  2022-03-28  /boxscores/202203280IND.html  ATL  /teams/ATL/2022.html   
75    76  2022-03-30  /boxscores/202203300OKC.html  ATL  /teams/ATL/2022.html   
76    77  2022-03-31  /boxscores/202203310ATL.html  ATL  /teams/ATL/2022.html   
77    78  2022-04-02  /boxscores/202204020ATL.html  ATL  /teams/ATL/2022.html   
78    79  2022-04-05  /boxscores/202204050TOR.html  ATL  /teams/ATL/2022.html   
79    80  2022-04-06  /boxscores/202204060ATL.html  ATL  /teams/ATL/2022.html   
80    81  2022-04-08  /boxscores/202204080MIA.html  ATL  /teams/ATL/2022.html   
81    82  2022-04-10  /boxscores/202204100HOU.html  ATL  /teams/ATL/2022.html   

                                  Result           Advanced                 
   H/A  Opp           Opp_html_id    W/L   Tm  Opp     ORtg   DRtg   Pace   
0    1  DAL  /teams/DAL/2022.html      1  113   87    109.9   84.6  102.9  \
1    0  CLE  /teams/CLE/2022.html      0   95  101     99.3  105.6   95.7   
2    1  DET  /teams/DET/2022.html      1  122  104    123.1  105.0   99.1   
3    0  NOP  /teams/NOP/2022.html      1  102   99    110.2  107.0   92.5   
4    0  WAS  /teams/WAS/2022.html      0  111  122    111.9  122.9   99.2   
5    0  PHI  /teams/PHI/2022.html      0   94  122     95.6  124.1   98.3   
6    1  WAS  /teams/WAS/2022.html      1  118  111    126.3  118.8   93.4   
7    0  BRK  /teams/BRK/2022.html      0  108  117    108.5  117.5   99.6   
8    1  UTA  /teams/UTA/2022.html      0   98  116    103.2  122.1   95.0   
9    0  PHO  /teams/PHO/2022.html      0  117  121    114.2  118.1  102.4   
10   0  GSW  /teams/GSW/2022.html      0  113  127    108.9  122.4  103.8   
11   0  UTA  /teams/UTA/2022.html      0   98  110    106.5  119.6   92.0   
12   0  DEN  /teams/DEN/2022.html      0   96  105    102.6  112.2   93.6   
13   1  MIL  /teams/MIL/2022.html      1  120  100    125.6  104.7   95.6   
14   1  ORL  /teams/ORL/2022.html      1  129  111    127.6  109.8  101.1   
..  ..  ...         

In [19]:
TGL_DF.drop('Match',level=0,axis=1)

Result           Advanced                                                  
      W/L   Tm  Opp     ORtg   DRtg   Pace    FTr   3PAr    TS%  TRB%  AST%   
0       1  113   87    109.9   84.6  102.9  0.096  0.372  0.577  52.4  68.9  \
1       0   95  101     99.3  105.6   95.7  0.152  0.343  0.450  50.0  52.6   
2       1  122  104    123.1  105.0   99.1  0.233  0.356  0.615  57.6  52.2   
3       1  102   99    110.2  107.0   92.5  0.177  0.313  0.493  55.6  52.5   
4       0  111  122    111.9  122.9   99.2  0.159  0.239  0.589  45.7  54.2   
5       0   94  122     95.6  124.1   98.3  0.200  0.232  0.455  57.0  66.7   
6       1  118  111    126.3  118.8   93.4  0.349  0.410  0.616  56.6  63.2   
7       0  108  117    108.5  117.5   99.6  0.160  0.372  0.537  55.3  56.1   
8       0   98  116    103.2  122.1   95.0  0.317  0.341  0.524  41.0  51.4   
9       0  117  121    114.2  118.1  102.4  0.237  0.340  0.546  54.3  42.9   
10      0  113  127    108.9  122.4  103.8  0.317  0.524  0.605  47.3  55.3   
11      0   98  110    106.5  119.6   92.0  0.118  0.412  0.548  42.1  59.5   
12      0   96  105    102.6  112.2   93.6  0.226  0.301  0.469  50.5  52.6   
13      1  120  100    125.6  104.7   95.6  0.134  0.361  0.584  63.0  44.7   
14      1  129  111    127.6  109.8  101.1  0.376  0.353  0.651  52.9  68.1   
..    ...  ...  ...      ...    ...    ...    ...    ...    ...   ...   ...   
67      1  122  113    116.2  107.7  105.0  0.294  0.440  0.496  46.7  70.0   
68      0  106  116    112.8  123.4   94.0  0.151  0.398  0.534  50.0  70.0   
69      1  120  105    122.7  107.4   97.8  0.231  0.308  0.599  45.5  62.2   
70      0  112  117    108.8  113.6  103.0  0.264  0.368  0.577  42.6  53.7   
71      1  117  111    123.6  117.3   94.7  0.333  0.444  0.630  51.2  65.0   
72      0  101  122     98.7  119.3  102.3  0.163  0.478  0.512  42.6  56.8   
73      1  121  110    126.6  115.0   95.6  0.213  0.416  0.621  48.4  58.7   
74      1  132  123    138.3  128.9   95.4  0.302  0.407  0.677  58.8  68.1   
75      1  136  118    129.5  112.4  105.0  0.242  0.429  0.675  50.6  57.1   
76      1  131  107    133.7  109.2   98.0  0.308  0.429  0.634  50.5  60.0   
77      1  122  115    116.4  109.7  104.8  0.570  0.477  0.567  50.9  50.0   
78      0  108  118    112.3  122.7   96.1  0.128  0.394  0.544  41.7  72.1   
79      1  118  103    121.4  106.0   97.2  0.207  0.424  0.588  47.0  60.5   
80      0  109  113    114.4  118.6   95.3  0.216  0.386  0.566  56.6  40.5   
81      1  130  114    137.4  120.5   94.6  0.276  0.437  0.666  59.5  64.4   

               Offensive Four Factors                      
    STL%  BLK%                   eFG%  TOV%  ORB% FT/FGA   
0    7.8  18.0                  0.559  11.7  13.0  0.085  \
1    5.2   4.9                  0.434   7.9  28.8  0.091   
2   11.1   5.2                  0.578  11.6  28.6  0.200   
3    4.3   7.0                  0.458   9.6  37.5  0.146   
4    4.0   4.6                  0.580  12.1  14.0  0.102   
5    6.1   2.1                  0.421  15.5  38.5  0.147   
6    9.6  10.6                  0.536  10.3  31.0  0.349   
7    8.0  10.0                  0.505  12.2  25.5  0.138   
8   11.6  10.0                  0.470   8.8  11.6  0.256   
9    2.9   2.2                  0.500  10.1  27.5  0.206   
10   5.8   4.2                  0.567  16.2   9.4  0.244   
11   6.5  11.6                  0.541  11.0  14.3  0.071   
12   2.1   9.1                  0.435   7.3  26.4  0.161   
13   6.3   2.3                  0.562  10.5  36.6  0.113   
14   9.9  13.5                  0.635  10.0  23.7  0.247   
..   ...   ...                    ...   ...   ...    ...   
67  12.4   9.4                  0.422   3.1  27.4  0.275   
68   6.4   3.8                  0.500  10.8  27.3  0.140   
69  12.3   9.4                  0.566   8.2  17.5  0.187   
70   6.8   5.3                  0.529  14.1  16.3  0.230   
71   7.4   9.6                  0.586  13.1  27.0  0.272   
72   7.8   5.5 

In [3]:
# Get the difference between two consecutive rows in the date column
# Convert the column to datetime first
TGL_INFO_MDATE_DF = pd.to_datetime(TGL_DF[('Match','Date')])
TGL_INFO_DAYS_BTW_2GM = TGL_INFO_MDATE_DF.diff().dt.days.value_counts(dropna=False)
TGL_INFO_DAYS_BTW_3GM = TGL_INFO_MDATE_DF.diff(2).dt.days.value_counts(dropna=False)
TGL_INFO_DAYS_BTW_2GM.sort_index(), TGL_INFO_DAYS_BTW_3GM.sort_index()

((Match, Date)
 1.0    16
 2.0    49
 3.0    12
 4.0     2
 5.0     1
 8.0     1
 NaN     1
 Name: count, dtype: int64,
 (Match, Date)
 3.0     23
 4.0     31
 5.0     20
 6.0      2
 7.0      2
 9.0      1
 10.0     1
 NaN      2
 Name: count, dtype: int64)

In [4]:
# Get the Result, Team and Opponent columns
TM_GL_STATS_DF = TGL_DF[['Result','Team','Opponent']]
# Get the cumulative average of the dataframe
TM_GL_STATS_DF

Result           Team                                                     
      W/L   Tm  Opp   FG  FGA    FG%  3P 3PA    3P%  FT FTA    FT% ORB TRB   
0       1  113   87   45   94  0.479  15  35  0.429   8   9  0.889   6  55  \
1       0   95  101   38   99  0.384  10  34  0.294   9  15  0.600  17  54   
2       1  122  104   46   90  0.511  12  32  0.375  18  21  0.857  10  49   
3       1  102   99   40   96  0.417   8  30  0.267  14  17  0.824  21  55   
4       0  111  122   48   88  0.545   6  21  0.286   9  14  0.643   6  43   
5       0   94  122   36   95  0.379   8  22  0.364  14  19  0.737  20  49   
6       1  118  111   38   83  0.458  13  34  0.382  29  29  1.000  13  47   
7       0  108  117   41   94  0.436  13  35  0.371  13  15  0.867  12  52   
8       0   98  116   35   82  0.427   7  28  0.250  21  26  0.808   5  32   
9       0  117  121   42   97  0.433  13  33  0.394  20  23  0.870  14  50   
10      0  113  127   38   82  0.463  17  43  0.395  20  26  0.769   3  35   
11      0   98  110   37   85  0.435  18  35  0.514   6  10  0.600   6  32   
12      0   96  105   38   93  0.409   5  28  0.179  15  21  0.714  14  50   
13      1  120  100   47   97  0.485  15  35  0.429  11  13  0.846  15  51   
14      1  129  111   47   85  0.553  14  30  0.467  21  32  0.656   9  46   
..    ...  ...  ...  ...  ...    ...  ..  ..    ...  ..  ..    ...  ..  ..   
67      1  122  113   40  109  0.367  12  48  0.250  30  32  0.938  17  49   
68      0  106  116   40   93  0.430  13  37  0.351  13  14  0.929  12  43   
69      1  120  105   45   91  0.495  13  28  0.464  17  21  0.810   7  40   
70      0  112  117   41   87  0.471  10  32  0.313  20  23  0.870   7  40   
71      1  117  111   40   81  0.494  15  36  0.417  22  27  0.815  10  43   
72      0  101  122   37   92  0.402  14  44  0.318  13  15  0.867   9  40   
73      1  121  110   46   89  0.517  14  37  0.378  15  19  0.789  12  44   
74      1  132  123   47   86  0.547  17  35  0.486  21  26  0.808  13  40   
75      1  136  118   49   91  0.538  18  39  0.462  20  22  0.909   6  43   
76      1  131  107   45   91  0.495  17  39  0.436  24  28  0.857  12  48   
77      1  122  115   36   86  0.419  13  41  0.317  37  49  0.755  10  54   
78      0  108  118   43   94  0.457  14  37  0.378   8  12  0.667  10  43   
79      1  118  103   43   92  0.467  16  39  0.410  16  19  0.842   7  39   
80      0  109  113   42   88  0.477   8  34  0.235  17  19  0.895  13  43   
81      1  130  114   45   87  0.517  21  38  0.553  19  24  0.792  13  50   

                       Opponent                                                 
   AST STL BLK TOV  PF       FG  FGA    FG%  3P 3PA    3P%  FT FTA    FT% ORB   
0   31   8   9  13  16       31   93  0.333  13  43  0.302  12  13  0.923  10  \
1   20   5   3   9  23       37   89  0.416   7  28  0.250  20  27  0.741  12   
2   24  11   3  13  19       40   91  0.440   9  33  0.273  15  18  0.833  11   
3   21   4   4  11  14       40   93  0.430  11  36  0.306   8   9  0.889   9   
4   26   4   3  13  16       46   99  0.465  13  34  0.382  17  21  0.810  14   
5   24   6   1  19  17       46   86  0.535  12  38  0.316  18  19  0.947   5   
6   24   9   5  11  17       41   86  0.477  13  39  0.333  16  16  1.000   7   
7   23   8   4  14  16       43   88  0.489  22  48  0.458   9  12  0.750   7   
8   18  11   4   9  24       41   81  0.506  17  41  0.415  17  23  0.739   8   
9   18   3   1  12  16       45   88  0.511  14  43  0.326  17  21  0.810   5   
10  21   6   2  18  16       44   92  0.478  18  44  0.409  21  22  0.955  10   
11  22   6   5  11  18       41   81  0.506  15  38  0.395  13  16  0.813   8   
12  20   2   5   8  20       40   90  0.444  10  35  0.286  15  20  0.750  10   
13  21   6   1  12  19       38   84  0.452  14  41  0.341  10  16  0.625   4   
14  32  10   7  11  17       43   95  0.453  16  43  0.372   9  14  0.643  12   
..  ..  ..  ..  ..  ..      ...  ...    ...  ..  ..

In [5]:
# Team rolling average
TM_GL_STATS_EXPD_AVG_DF = TM_GL_STATS_DF.expanding().mean()
TM_GL_STATS_EXPD_AVG_DF

Result                               Team                        
         W/L          Tm         Opp         FG        FGA       FG%   
0   1.000000  113.000000   87.000000  45.000000  94.000000  0.479000  \
1   0.500000  104.000000   94.000000  41.500000  96.500000  0.431500   
2   0.666667  110.000000   97.333333  43.000000  94.333333  0.458000   
3   0.750000  108.000000   97.750000  42.250000  94.750000  0.447750   
4   0.600000  108.600000  102.600000  43.400000  93.400000  0.467200   
5   0.500000  106.166667  105.833333  42.166667  93.666667  0.452500   
6   0.571429  107.857143  106.571429  41.571429  92.142857  0.453286   
7   0.500000  107.875000  107.875000  41.500000  92.375000  0.451125   
8   0.444444  106.777778  108.777778  40.777778  91.222222  0.448444   
9   0.400000  107.800000  110.000000  40.900000  91.800000  0.446900   
10  0.363636  108.272727  111.545455  40.636364  90.909091  0.448364   
11  0.333333  107.416667  111.416667  40.333333  90.416667  0.447250   
12  0.307692  106.538462  110.923077  40.153846  90.615385  0.444308   
13  0.357143  107.500000  110.142857  40.642857  91.071429  0.447214   
14  0.400000  108.933333  110.200000  41.066667  90.666667  0.454267   
..       ...         ...         ...        ...        ...       ...   
67  0.500000  112.941176  112.102941  41.205882  88.132353  0.469221   
68  0.492754  112.840580  112.159420  41.188406  88.202899  0.468652   
69  0.500000  112.942857  112.057143  41.242857  88.242857  0.469029   
70  0.492958  112.929577  112.126761  41.239437  88.225352  0.469056   
71  0.500000  112.986111  112.111111  41.222222  88.125000  0.469403   
72  0.493151  112.821918  112.246575  41.164384  88.178082  0.468479   
73  0.500000  112.932432  112.216216  41.229730  88.189189  0.469135   
74  0.506667  113.186667  112.360000  41.306667  88.160000  0.470173   
75  0.513158  113.486842  112.434211  41.407895  88.197368  0.471066   
76  0.519481  113.714286  112.363636  41.454545  88.233766  0.471377   
77  0.525641  113.820513  112.397436  41.384615  88.205128  0.470705   
78  0.518987  113.746835  112.468354  41.405063  88.278481  0.470532   
79  0.525000  113.800000  112.350000  41.425000  88.325000  0.470488   
80  0.518519  113.740741  112.358025  41.432099  88.320988  0.470568   
81  0.524390  113.939024  112.378049  41.475610  88.304878  0.471134   

                                                                                
           3P        3PA       3P%         FT        FTA       FT%        ORB   
0   15.000000  35.000000  0.429000   8.000000   9.000000  0.889000   6.000000  \
1   12.500000  34.500000  0.361500   8.500000  12.000000  0.744500  11.500000   
2   12.333333  33.666667  0.366000  11.666667  15.000000  0.782000  11.000000   
3   11.250000  32.750000  0.341250  12.250000  15.500000  0.792500  13.500000   
4   10.200000  30.400000  0.330200  11.600000  15.200000  0.762600  12.000000   
5    9.833333  29.000000  0.335833  12.000000  15.833333  0.758333  13.333333   
6   10.285714  29.714286  0.342429  14.428571  17.714286  0.792857  13.285714   
7   10.625000  30.375000  0.346000  14.250000  17.375000  0.802125  13.125000   
8   10.222222  30.111111  0.335333  15.000000  18.333333  0.802778  12.222222   
9   10.500000  30.400000  0.341200  15.500000  18.800000  0.809500  12.400000   
10  11.090909  31.545455  0.346091  15.909091  19.454545  0.805818  11.545455   
11  11.666667  31.833333  0.360083  15.083333  18.666667  0.788667  11.083333   
12  11.153846  31.538462  0.346154  15.076923  18.846154  0.782923  11.307692   
13  11.428571  31.785714  0.352071  14.785714  18.428571  0.787429  11.571429   
14  11.600000  31.666667  0.359733  15.200000  19.333333  0.778667  11.400000   
..        ...        ...       ...        ...        ...       ...        ...   
67  12.544118  33.897059  0.367132  17.985294  22.220588  0.808235  10.029412   
68  12.550725  33.942029  0.366899  17.913043  22.101449  0.809986  10.057971   
69  12.55

In [6]:
# Team last 10 games rolling average
TM_GL_STATS_ROLL_10_AVG_DF = TM_GL_STATS_DF.rolling(10).mean()
TM_GL_STATS_ROLL_10_AVG_DF

Result                Team                                                 
      W/L     Tm    Opp    FG   FGA     FG%    3P   3PA     3P%    FT   FTA   
0     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN  \
1     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN   
2     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN   
3     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN   
4     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN   
5     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN   
6     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN   
7     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN   
8     NaN    NaN    NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN   NaN   
9     0.4  107.8  110.0  40.9  91.8  0.4469  10.5  30.4  0.3412  15.5  18.8   
10    0.3  107.8  114.0  40.2  90.6  0.4453  10.7  31.2  0.3378  16.7  20.5   
11    0.3  108.1  114.9  40.1  89.2  0.4504  11.5  31.3  0.3598  16.4  20.0   
12    0.2  105.5  115.0  39.3  89.5  0.4402  10.8  30.9  0.3402  16.1  20.0   
13    0.2  107.3  115.1  40.0  89.6  0.4470  11.5  31.4  0.3564  15.8  19.6   
14    0.3  109.1  114.0  39.9  89.3  0.4478  12.3  32.3  0.3745  17.0  21.4   
..    ...    ...    ...   ...   ...     ...   ...   ...     ...   ...   ...   
67    0.6  117.0  114.1  42.1  90.0  0.4722  11.9  35.9  0.3307  20.9  25.4   
68    0.6  116.8  114.5  42.0  89.6  0.4729  12.4  36.8  0.3372  20.4  24.1   
69    0.6  116.1  115.0  41.7  90.4  0.4646  12.4  36.5  0.3417  20.3  24.0   
70    0.6  117.5  116.0  42.2  90.8  0.4683  12.7  36.6  0.3504  20.4  23.9   
71    0.6  116.2  114.7  41.8  90.5  0.4653  12.4  36.2  0.3471  20.2  23.8   
72    0.5  114.6  115.5  41.7  90.6  0.4637  12.5  37.0  0.3428  18.7  21.9   
73    0.6  115.7  115.2  42.1  90.2  0.4702  12.8  36.9  0.3517  18.7  21.6   
74    0.7  117.4  115.1  42.5  90.2  0.4749  13.4  36.6  0.3714  19.0  22.0   
75    0.7  119.8  116.3  43.1  90.3  0.4809  14.3  37.4  0.3886  19.3  22.3   
76    0.7  119.8  114.2  43.0  91.0  0.4756  14.3  37.5  0.3875  19.5  22.7   
77    0.7  119.8  114.4  42.6  88.7  0.4808  14.4  36.8  0.3942  20.2  24.4   
78    0.7  120.0  114.6  42.9  88.8  0.4835  14.5  36.8  0.3969  19.7  24.2   
79    0.7  119.8  114.4  42.7  88.9  0.4807  14.8  37.9  0.3915  19.6  24.0   
80    0.7  119.5  114.0  42.8  89.0  0.4813  14.6  38.1  0.3837  19.3  23.6   
81    0.7  120.8  114.3  43.3  89.6  0.4836  15.2  38.3  0.3973  19.0  23.3   

                                                   Opponent                 
       FT%   ORB   TRB   AST  STL  BLK   TOV    PF       FG   FGA     FG%   
0      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN  \
1      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN   
2      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN   
3      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN   
4      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN   
5      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN   
6      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN   
7      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN   
8      NaN   NaN   NaN   NaN  NaN  NaN   NaN   NaN      NaN   NaN     NaN   
9   0.8095  12.4  48.6  22.9  6.9  3.7  12.4  17.8     41.0  89.4  0.4602   
10  0.7975  12.1  46.6  21.9  6.7  3.0  12.9  17.8     42.3  89.3  0.4747   
11  0.7975  11.0  44.4  22.1  6.8  3.2  13.1  17.3     42.7  88.5  0.4837   
12  0.7832  11.4  44.5  21.7  5.9  3.4  12.6  17.4     42.7  88.4  0.4841   
13  0.7854  10.8  44.1  21.7  6.1  3.1  12.7  17.9     42.5  87.5  0.4863   
14  0.7867  11.1  44.4  22.3  6.7  3.5  12.5  18.0     42.2  87.1  0.4851   
..     ...   ...   ...   ...  ...  ...   ...   ...      ...   ...     ...   
67  0.820

In [7]:
# Team last 10 games rolling average
TM_GL_STATS_ROLL_20_AVG_DF = TM_GL_STATS_DF.rolling(20).mean()
TM_GL_STATS_ROLL_20_AVG_DF

Result                   Team                                          
      W/L      Tm     Opp     FG    FGA      FG%     3P    3PA      3P%   
0     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN  \
1     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
2     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
3     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
4     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
5     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
6     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
7     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
8     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
9     NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
10    NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
11    NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
12    NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
13    NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
14    NaN     NaN     NaN    NaN    NaN      NaN    NaN    NaN      NaN   
..    ...     ...     ...    ...    ...      ...    ...    ...      ...   
67   0.55  116.70  114.45  42.60  89.65  0.47875  12.50  34.80  0.35705   
68   0.50  115.55  114.20  42.15  90.10  0.47110  12.65  35.10  0.35845   
69   0.55  116.55  114.15  42.55  90.60  0.47300  12.70  34.90  0.36290   
70   0.50  115.95  114.25  42.40  90.75  0.47035  12.20  34.50  0.35355   
71   0.55  116.10  113.55  42.25  90.15  0.47195  12.50  34.85  0.35890   
72   0.55  116.45  114.50  42.25  90.00  0.47260  12.95  35.80  0.36480   
73   0.55  115.85  114.40  41.95  89.95  0.46955  12.80  36.00  0.35795   
74   0.60  116.40  113.75  41.95  89.10  0.47410  12.95  35.60  0.36595   
75   0.65  118.45  114.40  42.70  89.00  0.48270  13.25  35.65  0.37325   
76   0.65  118.80  113.95  42.80  89.55  0.48055  13.40  35.95  0.37385   
77   0.65  118.40  114.25  42.35  89.35  0.47650  13.15  36.35  0.36245   
78   0.65  118.40  114.55  42.45  89.20  0.47820  13.45  36.80  0.36705   
79   0.65  117.95  114.70  42.20  89.65  0.47265  13.60  37.20  0.36660   
80   0.65  118.50  115.00  42.50  89.90  0.47480  13.65  37.35  0.36705   
81   0.65  118.50  114.50  42.55  90.05  0.47445  13.80  37.25  0.37220   

                                                                           
       FT    FTA      FT%    ORB    TRB    AST   STL   BLK    TOV     PF   
0     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN  \
1     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
2     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
3     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
4     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
5     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
6     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
7     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
8     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
9     NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
10    NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
11    NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
12    NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
13    NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
14    NaN    NaN      NaN    NaN    NaN    NaN   NaN   NaN    NaN    NaN   
..    ...    ...      ...    ...    ...    ...   ...   ...    ...    ...   
67  19.00  23.10  0.83005  10.50  43.35  25.25  7.25  4.15  10.60  19.40   
68  18.60  22.20  0.84370  10.70  43.25  25.10  7.20  4.25  10.60  19.50   
69  18.

# 

In [33]:
TGL_DF['Result'][['Tm','Opp']]

,Tm,Opp
0,113,87
1,95,101
2,122,104
3,102,99
4,111,122
5,94,122
6,118,111
7,108,117
8,98,116
9,117,121


In [35]:
TGL_DF['Result'][['Tm','Opp']].apply(lambda x: x.iloc[0] - x.iloc[1],axis=1)

0     26
1     -6
2     18
3      3
4    -11
5    -28
6      7
7     -9
8    -18
9     -4
10   -14
11   -12
12    -9
13    20
14    18
      ..
67     9
68   -10
69    15
70    -5
71     6
72   -21
73    11
74     9
75    18
76    24
77     7
78   -10
79    15
80    -4
81    16
Length: 82, dtype: int64